TODO
- Plot lengths, sizes etc. of **random** videos
- Get article, cameo etc. of those videos and figure out if its applicable
- **Maybe youtube has a classification!**

In [68]:
import io
import os
from collections import Counter
import base64
import glob
from IPython.display import HTML
import psycopg2
import pandas as pd
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

from src import util
from src.data import gdelt_event_2
from src.data.cameo import CAMEO

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
conn = psycopg2.connect(database="gdelt_social_video", user="postgres")
c = conn.cursor()

platform = "youtube"
resolution = "lowest_res"
interesting_video_idx = 15

video_path = os.environ["DATA_PATH"] + "/raw/videos/%s/%s/random1000/" % (platform, resolution)
videos = glob.glob(video_path + "*.mp4")
interesting_video = videos[interesting_video_idx]
video_id = interesting_video.split("/")[-1].split(".")[0]

In [87]:
# Get all articles containing that video
articles = pd.read_sql_query("SELECT * from found_videos WHERE platform='%s' AND video_url LIKE '%%%s%%'"\
                             % (platform, video_id), con=conn)
articles.head()

,website_url,platform,video_url,hostname
0,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037,youtube,https://www.youtube.com/embed/0JrEwOqPogo,www.thedailystar.net


In [88]:
# Get all mentions that have that article's url as source_url
mentions = pd.DataFrame()
for index, row in articles.iterrows():
    mentions = mentions.append(pd.read_sql_query("SELECT * from all_mentions WHERE mention_identifier='%s'"\
                             % (row["website_url"]), con=conn), sort=False)
mentions.head()

,global_event_id,event_time_date,mention_time_date,mention_type,mention_source_name,mention_identifier,sentence_id,actor1_char_offset,actor2_char_offset,action_char_offset,in_raw_text,confidence,mention_doc_len,mention_doc_tone
0,768931514,20180702191500,20180702191500,1,thedailystar.net,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037,32,7735,-1,7745,True,100.0,8528,-6.444605
1,768931461,20180702191500,20180702191500,1,thedailystar.net,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037,33,7845,7872,7883,True,100.0,8528,-6.444605
2,768930694,20180702191500,20180702191500,1,thedailystar.net,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037,18,4555,4603,4590,True,100.0,8528,-6.444605
3,768930688,20180702191500,20180702191500,1,thedailystar.net,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037,10,2887,2868,2895,False,40.0,8528,-6.444605
4,768930684,20180702191500,20180702191500,1,thedailystar.net,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037,21,4953,-1,4978,True,100.0,8528,-6.444605


In [89]:
# Get all events that have that mention's url as source_url
events = pd.DataFrame()
for index, row in mentions.iterrows():
    url = row["mention_identifier"]
    if util.is_url(url): # Ignore identifiers that are from non-website sources (very few) 
                            # TODO pie chart: Most common hosts, other hosts, other types of sources
        events = events.append(pd.read_sql_query("SELECT * from all_export WHERE source_url='%s'"\
                                 % (url), con=conn))
events.head()

,global_event_id,sql_date,month_year,year,fraction_date,actor1_code,actor1_name,actor1_country_code,actor1_known_group_code,actor1_ethnic_code,actor1_religion1_code,actor1_religion2_code,actor1_type1_code,actor1_type2_code,actor1_type3_code,actor2_code,actor2_name,actor2_country_code,actor2_known_group_code,actor2_ethnic_code,actor2_religion1_code,actor2_religion2_code,actor2_type1_code,actor2_type2_code,actor2_type3_code,is_root_event,event_code,event_base_code,event_root_code,quad_class,goldstein_scale,num_mentions,num_sources,num_articles,avg_tone,actor1_geo_type,actor1_geo_full_name,actor1_geo_country_code,actor1_geo_adm1_code,actor1_geo_adm2_code,actor1_geo_lat,actor1_geo_long,actor1_geo_feature_id,actor2_geo_type,actor2_geo_fullname,actor2_geo__country_code,actor2_geo_adm1_code,actor2_geo_adm2_code,actor2_geo_lat,actor2_geo_long,actor2_geo_feature_id,action_geo_type,action_geo_full_name,action_geo_country_code,action_geo_adm1_code,action_geo_adm2_code,action_geo_lat,action_geo_long,action_geo_feature_id,date_added,source_url
0,768931514,20180702,201807,2018,2018.4986,OPP,ACTIVIST,None,None,None,None,None,OPP,None,None,None,None,None,None,None,None,None,None,None,None,False,012,012,01,1,-0.4,10,1,10,-6.444605,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,0,None,None,None,None,None,None,None,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,20180702191500,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
1,768931461,20180702,201807,2018,2018.4986,NGODEV,TRANSPARENCY INTERNATIONAL,None,None,None,None,None,NGO,DEV,None,BGD,BANGLADESH,BGD,None,None,None,None,None,None,None,False,012,012,01,1,-0.4,10,1,10,-6.444605,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,20180702191500,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
2,768930694,20180702,201807,2018,2018.4986,EDU,UNIVERSITY,None,None,None,None,None,EDU,None,None,EDUGOV,UNIVERSITY,None,None,None,None,None,EDU,GOV,None,False,010,010,01,1,0.0,15,1,10,-6.444605,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,20180702191500,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
3,768930688,20180702,201807,2018,2018.4986,EDU,COLLEGE,None,None,None,None,None,EDU,None,None,BGD,BANGLADESH,BGD,None,None,None,None,None,None,None,False,112,112,11,3,-2.0,4,1,4,-6.444605,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,20180702191500,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
4,768930684,20180702,201807,2018,2018.4986,EDU,UNIVERSITY,None,None,None,None,None,EDU,None,None,None,None,None,None,None,None,None,None,None,None,False,114,114,11,3,-2.0,10,1,10,-6.444605,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,0,None,None,None,None,None,None,None,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,5778,23.7231,90.4086,-2737683,20180702191500,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037


In [90]:
# Extract all CAMEO codes in all articles that also have that video
interesting_columns = ["global_event_id", 
                       
                       "event_code", 
                       "event_base_code",
                       "event_root_code",
                       "quad_class", 
                       "source_url"]
events[interesting_columns].head()

,global_event_id,event_code,event_base_code,event_root_code,quad_class,source_url
0,768931514,012,012,01,1,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
1,768931461,012,012,01,1,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
2,768930694,010,010,01,1,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
3,768930688,112,112,11,3,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037
4,768930684,114,114,11,3,https://www.thedailystar.net/frontpage/bcl-men-action-again-1599037


In [91]:
# TODO this still contains duplicates
"The video %s is on %d pages that mention a total of %d events" % (video_id, len(mentions), len(events))

'The video 0JrEwOqPogo is on 10 pages that mention a total of 80 events'

In [92]:
Counter(events["event_code"]) # The actual unique events are very few

Counter({'012': 20, '010': 10, '112': 10, '114': 10, '011': 30})

In [93]:
for event_id in set(events["event_code"]):
    print(event_id + ": " + CAMEO.get_description(event_id))

011: MAKE PUBLIC STATEMENT > Decline comment
112: DISAPPROVE > Accuse, not specified below
114: DISAPPROVE > Complain officially
012: MAKE PUBLIC STATEMENT > Make pessimistic comment
010: MAKE PUBLIC STATEMENT > Make statement, not specified below


In [94]:
# Only relative paths can be used
rel_path = os.path.relpath(interesting_video, os.getcwd())
HTML("""
<video width="320" height="240" controls>
  <source src="%s" type="video/mp4">
</video>
""" % rel_path)